In [ ]:
def radiation_to_power(radiation, area, yield_r=0.175, pr=0.6, hours=1):
    """
    Convert solar radiation to power output in kWh.
    
    Parameters:
    - radiation: Solar radiation in Joules (J)
    - area: Area of the solar panel in square meters (m^2)
    - yield_r: Solar panel yield (default: 0.175)
    - pr: Performance ratio (default: 0.6)
    - hours: Duration in hours (default: 1)
    
    Returns:
    - kWh: Power output in kilowatt-hours (kWh)
    """
    kWh = radiation * area * yield_r * pr * hours * (1 / 3600000)
    return kWh

rad = 9e6  # Radiation in Joules
area = 100  # Area in square meters

power_kWh = radiation_to_power(rad, area)
power_kWh

In [ ]:
# Extract Class 1
import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape

input_raster = "Final_sum_reclass.tif"
output_shapefile = "Class_1.shp"

with rasterio.open(input_raster) as src:

    raster_data = src.read(1)
    transform = src.transform
    crs = src.crs

shapes_generator = shapes(raster_data, mask=(raster_data == 1), transform=transform)

geometries = [shape(geom) for geom, value in shapes_generator if value == 1]

gdf = gpd.GeoDataFrame(geometry=geometries, crs=crs.to_string())

gdf.to_file(output_shapefile, driver="ESRI Shapefile")

print(f"Extraction completed. Output saved at: {output_shapefile}")

In [ ]:
# Calculate area
import geopandas as gpd

input_shapefile = "Class_1.shp"
intermediate_shapefile = "Class_1_area.shp"
output_shapefile = "Class_1_area_WGS84.shp"

gdf = gpd.read_file(input_shapefile)

print(f"Original CRS: {gdf.crs}")

# Choose a UTM zone that fits the most of Indonesia - UTM Zone 49S (EPSG:32750)
utm_crs = "EPSG:32750"
gdf = gdf.to_crs(utm_crs)

gdf["area_m2"] = gdf.geometry.area

gdf.to_file(intermediate_shapefile)

print(f"Area calculation completed. Output saved to: {intermediate_shapefile}")

for index, row in gdf.iterrows():
    print(f"Feature {index} area: {row['area_m2']:.2f} square meters")

# Reproject back to WGS84 (EPSG:4326)
gdf_wgs84 = gdf.to_crs("EPSG:4326")

gdf_wgs84.to_file(output_shapefile)

print(f"Reprojection completed. Output saved to: {output_shapefile}")

In [ ]:
# Calculate power
import geopandas as gpd
import xarray as xr
import numpy as np
from shapely.geometry import shape
from rasterio.features import shapes

radiation_file = "ssrd_mean.tif" 
area_shapefile = "Class_1_area_WGS84.shp"
output_shapefile = "Class_1_power.shp"

yield_r = 0.175
pr = 0.6
hours = 1
max_area = 12000000  # Maximum area limit (m²)

ds = xr.open_dataset(radiation_file)
ssrd_data = ds["band_data"]
ssrd_data = ssrd_data.squeeze()

if not ssrd_data.rio.crs:
    ssrd_data = ssrd_data.rio.write_crs("EPSG:4326")

radiation_array = ssrd_data.values
radiation_transform = ssrd_data.rio.transform()
radiation_crs = ssrd_data.rio.crs

results = (
    {"properties": {"value": value}, "geometry": shape(geometry)}
    for geometry, value in shapes(radiation_array, transform=radiation_transform)
    if value > 0
)
radiation_gdf = gpd.GeoDataFrame.from_features(results, crs=radiation_crs)

area_gdf = gpd.read_file(area_shapefile)

if area_gdf.crs != radiation_gdf.crs:
    area_gdf = area_gdf.to_crs(radiation_gdf.crs)

area_gdf = area_gdf[area_gdf.is_valid]

area_gdf["area_m2"] = area_gdf["area_m2"].apply(lambda x: min(x, max_area))

merged_gdf = gpd.sjoin(area_gdf, radiation_gdf, how="inner", predicate="intersects")

def calculate_power(row):
    return row["value"] * row["area_m2"] * yield_r * pr * hours / 3600000

merged_gdf["power_kWh"] = merged_gdf.apply(calculate_power, axis=1)

aggregated_df = merged_gdf.groupby("FID", as_index=False).agg({
    "area_m2": "first",
    "value": "mean",
    "power_kWh": "sum",
    "geometry": "first"   
})

aggregated_gdf = gpd.GeoDataFrame(aggregated_df, geometry="geometry", crs=area_gdf.crs)

aggregated_gdf = aggregated_gdf.rename(columns={
    "area_m2": "area_m2",
    "value": "radiation",
    "power_kWh": "pwr_kwh"
})

aggregated_gdf.to_file(output_shapefile)
print(f"Results saved to: {output_shapefile}")

In [ ]:
# shp to csv
import geopandas as gpd

input_shapefile = "Class_1_power.shp"

output_csv = "Class_1_power.csv"

gdf = gpd.read_file(input_shapefile)

gdf.to_csv(output_csv, index=False)

print(f"Shapefile successfully exported to CSV at: {output_csv}")